# Advanced Mapping
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/htcondor/htmap/master?urlpath=lab/tree/tutorials/advanced-mapping.ipynb)

So far we've built our maps using the top-level mapping functions.
These functions are useful for tutorials, but don't give us the full flexibility that we might need when working with arbitrary Python functions.
They're also sometimes inconvenient to use, especially if you don't like typing the names of your functions over and over.
The tools described in this tutorial fix those problems.

## Starmap

Back in [Working With Files](working-with-files.ipynb), we noted that [htmap.map](../api.rst#htmap.map) was only able to handle functions that took a single argument.
To work with a function that took two arguments, we needed to use [htmap.build_map](../api.rst#htmap.build_map) to build up the map inside a loop.

Sometimes, you don't want to loop.
[htmap.starmap](../api.rst#htmap.starmap) provides the flexibility to completely specify the positional and keyword arguments for every component without needing an explicit `for`-loop.

Unfortunately, that looks like this:

In [1]:
import htmap

def power(x, p = 1):
        return x ** p

In [2]:
starmap = htmap.starmap(
    func = power,
    args = [
        (1,),
        (2,),
        (3,),
    ],
    kwargs = [
        {'p': 1},
        {'p': 2},
        {'p': 3},
    ],
)

print(list(starmap))  # [1, 4, 27]

created map sweet-trite-tub with 3 components
[1, 4, 27]


A slightly more pleasant but less obvious way to construct the arguments would be like this:

In [3]:
starmap = htmap.starmap(
    func = power,
    args = ((x,) for x in range(1, 4)),
    kwargs = ({'p': p} for p in range(1, 4)),
)

print(list(starmap))  # [1, 4, 27]

created map soggy-light-jaw with 3 components
[1, 4, 27]


But that isn't really a huge improvement.
Sometimes you'll need the power and compactness of `starmap`, but we recommend [htmap.build_map](../api.rst#htmap.build_map) for general use.

## Mapped Functions

If you're tired of re-typing the name of your function all the time, create a [htmap.MappedFunction](../api.rst#htmap.MappedFunction) using the [htmap.mapped](../api.rst#htmap.mapped) decorator:

In [4]:
@htmap.mapped
def double(x):
    return 2 * x

print(double)

<MappedFunction(func = <function double at 0x7fbe681531e0>, map_options = {})>


The resulting `MappedFunction` has methods that correspond to all the mapping functions, but with the function already filled in.

For example:

In [5]:
doubled = double.map(range(10))

print(list(doubled))  # [0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

created map sweet-twin-jaw with 10 components
[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


The real utility of mapped functions is that they can carry default map options, which are **inherited** by any maps created from them.
For example, if we know that a certain function will always need a large amount of memory and disk space, we can specify it for **any** map like this:

In [6]:
@htmap.mapped(
    map_options = htmap.MapOptions(
        request_memory = '1GB',
        request_disk = '10GB',
    )
)
def big_array(_):
    big = list(range(1_000_000))  # imagine this is way bigger...
    return big

Now our `request_memory` and `request_disk` will be set for each map, without needing to specify it in the `MapOptions` of each individual `map` call.
We can still override the setting for a certain map by manually passing [htmap.MapOptions](../api.rst#htmap.MapOptions).

See [htmap.MapOptions](../api.rst#htmap.MapOptions) for some notes about how these inherited map options behave.

---
In the [next tutorial](error-handling.ipynb) we'll finally address the most important part of programming: what to do when things go wrong!